In [1]:
import requests
import datetime
import json
import pytz

from urllib.parse import urljoin 
from getpass import getpass
from requests.auth import HTTPBasicAuth
from pprint import pprint

The Ovation web API is a RESTful API that you use with any HTTP client. In this tutorial, we'll use the Python `requests` package.

### Utilities

In [2]:
API_HOST = 'https://api-dev.ovation.io/' #'http://127.0.0.1:3000'
def make_url(path, host=API_HOST):
    return urljoin(host, path)

In [3]:
make_url('/api/v1/projects')

'https://api-dev.ovation.io/api/v1/projects'

### Credentials

Enter your Ovation API key:

In [4]:
api_key = getpass(prompt="API Key: ")

API Key: ········


We create a `requests.Session` object for use with the Ovation API

In [5]:
def make_session(api_key=api_key):
    
    api = requests.Session()
    api.verify = True # Verify SSL certificates
    api.headers['content-type'] = 'application/json'
    api.headers['authorization'] = 'Bearer {}'.format(api_key)
    
    return api

api = make_session(api_key)

### Projects

In [6]:
make_url('/api/v1/projects')

'https://api-dev.ovation.io/api/v1/projects'

In [7]:
r = api.get(make_url('/api/v1/projects'))
r.raise_for_status()

In [8]:
projects = r.json()['projects']

In [9]:
len(projects)

13

In [10]:
pprint(projects[0])

{'_id': '3d058a54-837c-4b24-bf58-4b0bc09b4e76',
 '_rev': '3-64743c417f02ed110d2dc437189757ec',
 'api_version': 4,
 'attributes': {'comment': 'from-api!', 'name': 'demo-patient-1'},
 'links': {'_collaboration_roots': [],
           'files': {'related': '/api/v1/projects/3d058a54-837c-4b24-bf58-4b0bc09b4e76/links/files',
                     'self': '/api/v1/projects/3d058a54-837c-4b24-bf58-4b0bc09b4e76/links/files/relationships'},
           'folders': {'related': '/api/v1/projects/3d058a54-837c-4b24-bf58-4b0bc09b4e76/links/folders',
                       'self': '/api/v1/projects/3d058a54-837c-4b24-bf58-4b0bc09b4e76/links/folders/relationships'},
           'notes': '/notes',
           'properties': '/properties',
           'self': '/api/v1/projects/3d058a54-837c-4b24-bf58-4b0bc09b4e76',
           'tags': '/tags',
           'timeline-events': '/timeline-events'},
 'owner': '15cab930-1e24-0131-026c-22000a977b96',
 'type': 'Project'}


In [12]:
project = projects[1]
pprint(project)

{'_id': '52b6366a-1528-4bf9-a091-09c4e935d1cd',
 '_rev': '3-eec6d9c40436a57aca7d88e96f92cb54',
 'api_version': 4,
 'attributes': {'name': 'abby'},
 'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd'],
           'files': {'related': '/api/v1/projects/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/files',
                     'self': '/api/v1/projects/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/files/relationships'},
           'folders': {'related': '/api/v1/projects/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/folders',
                       'self': '/api/v1/projects/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/folders/relationships'},
           'notes': '/notes',
           'properties': '/properties',
           'self': '/api/v1/projects/52b6366a-1528-4bf9-a091-09c4e935d1cd',
           'tags': '/tags',
           'timeline-events': '/timeline-events'},
 'owner': '15cab930-1e24-0131-026c-22000a977b96',
 'type': 'Project'}


## Add a folder

To add a 'child' element, post a new entity descriptor to the parent. In this case, we create a `Folder` in the project by sending a `POST` to the `Project`

In [12]:
pprint(api.get(make_url("/api/v1/projects/540e18b1-8dde-4dee-ba77-097662e969e0")).json())

{'project': {'_id': '540e18b1-8dde-4dee-ba77-097662e969e0',
             '_rev': '1-c1ce2942b179ff126fffe0d6743a47e4',
             'api_version': 4,
             'attributes': {'label': 'speed test'},
             'links': {'_collaboration_roots': [],
                       'files': {'related': '/api/v1/projects/540e18b1-8dde-4dee-ba77-097662e969e0/links/files',
                                 'self': '/api/v1/projects/540e18b1-8dde-4dee-ba77-097662e969e0/links/files/relationships'},
                       'folders': {'related': '/api/v1/projects/540e18b1-8dde-4dee-ba77-097662e969e0/links/folders',
                                   'self': '/api/v1/projects/540e18b1-8dde-4dee-ba77-097662e969e0/links/folders/relationships'},
                       'notes': '/notes',
                       'properties': '/properties',
                       'self': '/api/v1/projects/540e18b1-8dde-4dee-ba77-097662e969e0',
                       'tags': '/tags',
                       'timeline-events':

In [13]:
folders = [{"type": "Folder", "attributes":{"label": "api demo"}}]
r = api.post(make_url("/api/v1/projects/{}".format(project['_id'])), json=folders)
r.raise_for_status()
pprint(r.json())

{'entities': [{'_id': '45f34b3d-0a07-491f-91d0-a29b518e26e0',
               'api_version': 4,
               'attributes': {'label': 'api demo'},
               'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd']},
               'owner': '15cab930-1e24-0131-026c-22000a977b96',
               'type': 'Folder'}],
 'links': [{'_id': '52b6366a-1528-4bf9-a091-09c4e935d1cd--folders-->45f34b3d-0a07-491f-91d0-a29b518e26e0',
            'inverse_rel': 'parents',
            'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd',
                                               '52b6366a-1528-4bf9-a091-09c4e935d1cd'],
                      'self': '/api/v1/relationships/52b6366a-1528-4bf9-a091-09c4e935d1cd--folders-->45f34b3d-0a07-491f-91d0-a29b518e26e0'},
            'rel': 'folders',
            'source_id': '52b6366a-1528-4bf9-a091-09c4e935d1cd',
            'target_id': '45f34b3d-0a07-491f-91d0-a29b518e26e0',
            'type': 'Relation',
         

In [14]:
folder = r.json()['entities'][0]
pprint(folder)

{'_id': '45f34b3d-0a07-491f-91d0-a29b518e26e0',
 'api_version': 4,
 'attributes': {'label': 'api demo'},
 'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd']},
 'owner': '15cab930-1e24-0131-026c-22000a977b96',
 'type': 'Folder'}


## Creating links

In [15]:
link_url = make_url('/api/v1/projects/{}/links/foo/relationships'.format(project['_id']))
links = [{'target_id': folder['_id']}]
r = api.post(link_url, json=links)

In [16]:
r.raise_for_status()
pprint(r.json())

{'entities': [{'_id': '52b6366a-1528-4bf9-a091-09c4e935d1cd',
               '_rev': '4-8c45666c0e498fabb4923bb83aab9750',
               'api_version': 4,
               'attributes': {'name': 'abby'},
               'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd'],
                         'files': {'related': '/api/v1/projects/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/files',
                                   'self': '/api/v1/projects/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/files/relationships'},
                         'folders': {'related': '/api/v1/projects/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/folders',
                                     'self': '/api/v1/projects/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/folders/relationships'},
                         'notes': '/notes',
                         'properties': '/properties',
                         'self': '/api/v1/projects/52b6366a-1528-4bf9-a091-09c4e935d1cd',
                         '

### Following links

In [31]:
r = api.get(make_url('/api/v1/projects/{}/links/folders'.format(project['_id'])))
r.raise_for_status()
targets = r.json()['folders']
pprint(targets)
folder = targets[0]

[{'_id': '45f34b3d-0a07-491f-91d0-a29b518e26e0',
  '_rev': '3-dab5a5774423fc0c54afa6f0081cf998',
  'api_version': 4,
  'attributes': {'label': 'api demo'},
  'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd'],
            'children': {'related': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/children',
                         'self': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/children/relationships'},
            'files': {'related': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/files',
                      'self': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/files/relationships'},
            'notes': '/notes',
            'parents': {'related': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/parents',
                        'self': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/parents/relationships'},
            'properties': '/properties',
            'self': '/api/v1/fol

In [18]:
r = api.get(make_url('/api/v1/projects/{}/links/foo'.format(project['_id'])))
r.raise_for_status()
targets = r.json()['foo']
pprint(targets)

[{'_id': '45f34b3d-0a07-491f-91d0-a29b518e26e0',
  '_rev': '3-dab5a5774423fc0c54afa6f0081cf998',
  'api_version': 4,
  'attributes': {'label': 'api demo'},
  'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd'],
            'children': {'related': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/children',
                         'self': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/children/relationships'},
            'files': {'related': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/files',
                      'self': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/files/relationships'},
            'notes': '/notes',
            'parents': {'related': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/parents',
                        'self': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/parents/relationships'},
            'properties': '/properties',
            'self': '/api/v1/fol

## Tags

### Add a tag

In [27]:
tags_url = make_url('/api/v1/entities/{}/annotations/tags'.format(folder['_id'])) #folder['links']
pprint(tags_url)

'https://api-dev.ovation.io/api/v1/entities/45f34b3d-0a07-491f-91d0-a29b518e26e0/annotations/tags'


In [28]:
r = api.post(tags_url, json=[{'tag': 'whoop'}])
r.raise_for_status()

### Get tags

In [29]:
r = api.get(tags_url)
r.raise_for_status()
pprint(r.json())

{'tags': {'45f34b3d-0a07-491f-91d0-a29b518e26e0': {'15cab930-1e24-0131-026c-22000a977b96': [{'_id': '03d477e0f13ca1b90e7e967fbf0610da',
                                                                                             '_rev': '1-02b11c19ab451d175b801fb4d3496188',
                                                                                             'annotation': {'tag': 'whoop'},
                                                                                             'annotation_type': 'tags',
                                                                                             'entity': '45f34b3d-0a07-491f-91d0-a29b518e26e0',
                                                                                             'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd']},
                                                                                             'type': 'Annotation',
                                                    

## File upload

### Create a file

Create a `File` in an existing folder

In [32]:
pprint(folder['links'])

{'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd'],
 'children': {'related': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/children',
              'self': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/children/relationships'},
 'files': {'related': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/files',
           'self': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/files/relationships'},
 'notes': '/notes',
 'parents': {'related': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/parents',
             'self': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0/links/parents/relationships'},
 'properties': '/properties',
 'self': '/api/v1/folders/45f34b3d-0a07-491f-91d0-a29b518e26e0',
 'tags': '/tags',
 'timeline-events': '/timeline-events'}


In [34]:
r = api.post(make_url(folder['links']['self']), json=[{'type': 'File',
                                                                 'attributes': {}}])
r.raise_for_status()
file = r.json()['entities'][0]
pprint(file)

In [55]:
file = api.get(make_url('/api/v1/files/{}'.format(file['_id']))).json()['file']
pprint(file)

{'_id': 'dc7ee82b-1d9e-447c-85bb-1b42af496f5f',
 '_rev': '2-9552620c4bf7ce05f6ee0b6b4d328662',
 'api_version': 4,
 'attributes': {},
 'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd'],
           'head': {'related': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f/links/head',
                    'self': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f/links/head/relationships'},
           'notes': '/notes',
           'parents': {'related': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f/links/parents',
                       'self': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f/links/parents/relationships'},
           'properties': '/properties',
           'revisions': {'related': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f/links/revisions',
                         'self': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f/links/revisions/relationships'},
           'self': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f',
   

In [53]:
revs_url = make_url(file['links']['revisions']['self'])
print(revs_url)

https://api-dev.ovation.io/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f/links/revisions/relationships


### Create a Revision

In [52]:
file_url = make_url(file['links']['self'])
print(file_url)

https://api-dev.ovation.io/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f


In [56]:
pprint(api.get(file_url).json())

{'file': {'_id': 'dc7ee82b-1d9e-447c-85bb-1b42af496f5f',
          '_rev': '2-9552620c4bf7ce05f6ee0b6b4d328662',
          'api_version': 4,
          'attributes': {},
          'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd'],
                    'head': {'related': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f/links/head',
                             'self': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f/links/head/relationships'},
                    'notes': '/notes',
                    'parents': {'related': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f/links/parents',
                                'self': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f/links/parents/relationships'},
                    'properties': '/properties',
                    'revisions': {'related': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42af496f5f/links/revisions',
                                  'self': '/api/v1/files/dc7ee82b-1d9e-447c-85bb-1b42a

In [48]:
r = api.post(make_url(file['links']['self']), json=[{'type': 'Revision',
                                                    'attributes': {'content_type': 'text/plain',
                                                                  'data_url': '',
                                                                  'file_name': 'demo.txt',
                                                                  'version': '',
                                                                  'previous': [],
                                                                  'resource': file['_id']}}])
r.raise_for_status()
rev = r.json()['revisions'][0]

HTTPError: 500 Server Error: Internal Server Error

Create a new `Resource` in Rails

In [41]:
rails = requests.Session()
rails.verify = True # Verify SSL certificates
rails.headers['content-type'] = 'application/json'
rails.headers['authorization'] = HTTPBasicAuth(api_key, 'X')

new_resource = {'entity_id': folder['_id'],
               'path': 'demo.txt'}
r = rails.post('https://ovation.io/api/v1/resources', json=new_resource)
r.raise_for_status()

rsrc = r.json()
pprint(rsrc)

TypeError: expected string or bytes-like object